In [ ]:
# 15-11 コメント
import random

# 親クラスの作成：プレイヤーとモンスターの共通部分を設定する
class Character:
    def __init__(self, name, HP, attack_power, MP, magic_power): #
        self.name   = name
        self.hp     = HP
        self.attack = attack_power
        self.mp = MP
        self.magic = magic_power
        self.max_hp = HP

    # ①プレイヤー/モンスターの攻撃と生存判定
                        # 　敵　 　味方                  最低HPを狙う
    def attack_start(self, enemys, allys, attack_message, strategy, players, monsters):
        random.shuffle(allys)  # 攻撃する順番をランダムにする
        for ally in allys:     # 順番に攻撃
            print(f"\n☆{ally.name}の番☆")
            print(ally.select_attack_type(enemys, allys, attack_message, strategy))

            # 攻撃後、生存者を確認し、どちらかが全滅していれば戦闘を終了
            enemys[:] = [enemy for enemy in enemys if enemy.hp > 0] # △攻撃された方の生存リスト
                # △[:]をつけることによって、下記のmonstersリスト or playersリストを更新してくれる
            allys[:]  = [ally for ally in allys if ally.hp > 0]     # △味方リスト

            if not monsters:
                print("\n！！敵をすべて倒した！！")
                return True
            elif not players:
                print("\n～ゲームオーバー～")
                return True
        return None


    # ②攻撃方法を選択（物理攻撃か他の攻撃か決める）
    def select_attack_type(self, attack_target, heal_target, attack_message, strategy):
        #action_type = int(input("1：攻撃、2：魔法　選択→"))
        attack_type = random.randint(1,2)
        if attack_type == 1:
            return self.common_select_attack_target(attack_target, attack_message, strategy)
        else:
            return self.attack_type2(attack_target, heal_target, attack_message, strategy)


    # ③-1物理攻撃の時はターゲットを決める
    def common_select_attack_target(self, attack_targets, attack_message, attack_strategy):
        # 生きているキャラをリストに入れる
        alive_list = [attack_target for attack_target in attack_targets if attack_target.hp > 0]
            # プレイヤーが攻撃の時は最もHPが低い敵を選ぶ
            if attack_strategy == "lowest":
                attack_target = min(alive_list, key=lambda attack_target: attack_target.hp)
            else: # モンスターが攻撃の時はランダムに攻撃相手を選ぶ
                    attack_target = random.choice(alive_list)
            return self.common_attack(attack_target, attack_message)


    # ③-2全員、物理攻撃
    def common_attack(self, attack_target, attack_message):
        # 会心・ミス・通常攻撃の3パターンからランダムに物理攻撃を行う
        physical_attack_list =  [  1,   2,   3]
        weights              =  [0.1, 0.2, 0.7]
        physical_attack_type = random.choices(physical_attack_list, weights)[0] # △重み（確率）を使いたい場合はchoices。返り値がリストなので [0] をつける

        if physical_attack_type == 1:
            damage = self.attack * 2
            damage_text = f"{self.name}の攻撃。会心の一撃！{attack_target.name}に{damage}のダメージ。"

        elif physical_attack_type == 2:
            damage = 0
            damage_text = f"{self.name}の攻撃。ミス！{attack_target.name}は攻撃を受けない。"

        else:
            damage = self.attack
            damage_text = f"{self.name}の攻撃。{attack_target.name}に{damage}のダメージ。"

        # 3.残りのHPの計算
        attack_target.hp -= damage

        # 4.物理攻撃用のテキスト作成
        if attack_target.hp <= 0 :
            attack_target.hp = 0
            return f"{damage_text}{attack_target.name}のHPは{attack_target.hp}。{attack_target.name}{attack_message}"
        else:
            return f"{damage_text}{attack_target.name}のHPは{attack_target.hp}。"
        # モンスター・プレイヤー共通、物理攻撃おわり



    # ④全員共通、攻撃魔法
    def common_magic(self, targets, attack_message, required_MP, default, magic, target_type):
        # 1.MP確認
        if self.mp < required_MP:         # MP確認。足りなかったら攻撃せずに終わる
            return f"{self.name}は{magic}を唱えた。MPが足りない！(現在MP:{self.mp})"
        else:
            self.mp -= required_MP        # MP減らす
            damage = self.magic * default # ダメージの計算
            magic_text = f"{self.name}は{magic}を唱えた。\n"

            # 2.単体攻撃 or 全体攻撃を判定
            # target_list = [random.choice(targets)] if target_type == "single" else targets
            if target_type == "single" :
                target_list = [random.choice(targets)] # 単体攻撃なら攻撃対象をランダムに選んでリストにいれる
            else:
                target_list = targets # 全体攻撃なら、敵全員をリストにいれる

            for target in target_list: # リストをループして、単体でも全体でも同じ処理をする
                target.hp -= damage

                if target.hp <= 0:
                    target.hp = 0  # 残りのHPを計算
                    magic_text += f"{target.name}に{damage}のダメージ。{target.name}のHPは{target.hp}。{target.name}{attack_message}\n"
                else:
                    magic_text += f"{target.name}に{damage}のダメージ。{target.name}のHPは{target.hp}。\n"

            return magic_text # 攻撃魔法用テキスト作成
            # モンスター・プレイヤー共通、攻撃魔法おわり



   # ⑤-1 回復魔法を使う場合、回復ターゲットを決める
    def common_select_heal_target(self, heal_targets, attack_message, heal_strategy,
                                  required_MP, default, magic, target_type):
        # 生きているプレイヤーをリストに入れる
        alive_list = [heal_target for heal_target in heal_targets if heal_target.hp > 0]
            # alive_targets = []
            # for heal_target in heal_targets:
            #   if heal_target.hp > 0:          # HPが0より大きい（生きてる）
            #       alive_targets.append(heal_target)

        if target_type == "single" : # 単体回復（ホイミ）で、
            #target = int(input("1：騎士、2：fire魔法使い（攻撃）、3：holy魔法使い（回復）、4：一番HPが低いプレーヤー　選択→"))

            if heal_strategy == "lowest": # プレイヤーが回復魔法を使う時は最もHPが低い味方を選ぶ
                heal_targets = [min(alive_list, key=lambda heal_target: heal_target.hp)]
                    # []をして、一人でもリストにする
                    # min(リスト, key=関数)で、alive_list内で値が最も小さい要素を返す
                    # key=lambda heal_target: heal_target.hp は、 heal_target一人一人のHPを比較して一番少ない人を選ぶ

            else: # モンスターはランダムに回復相手を選ぶ
                heal_targets = [random.choice(alive_list)]
        else:
            heal_targets = alive_list # 全体回復（ベホマラー）なら、生きている味方全員をリストにいれる
        return self.common_hoimi(heal_targets, attack_message, required_MP, default, magic)


    # ⑤-2全員、回復魔法
    def common_hoimi(self, heal_targets, attack_message, required_MP, default, magic):
        #　1.MP確認
        if self.mp < required_MP:         # MP確認。足りなかったら何もせずに終わる
            return f"{self.name}は{magic}を唱えた。MPが足りない！(現在MP:{self.mp})"
        else:
            self.mp -= required_MP        # MP減らす
            heal = self.magic * default   # 回復の計算
            heal_text =  f"{self.name}は{magic}を唱えた。\n"

            # 2.回復後のHP計算
            for heal_target in heal_targets:
                heal_target.hp += heal

                if heal_target.hp > heal_target.max_hp:
                    heal_target.hp = heal_target.max_hp

                # 3.回復用テキスト作成
                heal_text +=  f"{heal_target.name}のHPは{heal_target.hp}に回復した。\n"
            return heal_text
            # モンスター・プレイヤー共通、回復魔法おわり



# モンスタークラスの作成（Characterを継承しているので、Character 定義された共通のメソッドも使える）
class Monster(Character):
    pass

# 各モンスター固有の攻撃方法を定義
# モンスター1　メイジドラキー
class Doraki(Monster):
    # 単体に「ヒャド」を唱える魔法攻撃
    def attack_type2(self, attack_target, heal_target, attack_message, strategy):
        return self.common_magic(attack_target, attack_message, 3, 3, "ヒャド", "single")

# モンスター2　キメラ
class Kimera(Monster):
    # 全体に「ギラ」を唱える魔法攻撃
    def attack_type2(self, attack_target, heal_target, attack_message, strategy):
        return self.common_magic(attack_target, attack_message, 4, 5, "ギラ", "all")

# モンスター3　ホイミスライム
class h_suraimu(Monster):
    # 単体に「ホイミ」を唱える魔法回復を定義
    def attack_type2(self, attack_target, heal_target, attack_message, heal_strategy):
        return self.common_select_heal_target(heal_target, attack_message, heal_strategy,3, 6, "ホイミ", "single")
# モンスター達の攻撃処理おわり



# プレイヤークラスの作成
class Player(Character):
    pass

# 各プレイヤーの攻撃方法　3人
# 1.騎士
class Knight(Player):
    def attack_type2(self, attack_target, heal_target, attack_message, heal_strategy):
        # 単体に「ホイミ」を唱える魔法回復
        return self.common_select_heal_target(heal_target, attack_message, heal_strategy,3, 6, "ホイミ", "single")

# 2.fire魔法使い（攻撃）
class WizardA(Player):
    # 魔法攻撃(メラとイオ)か決める
    def attack_type2(self, attack_target, heal_target, attack_message, strategy):
        #magic_types = int(input("1：メラ、2：イオ　選択→"))
        magic_types = random.randint(1,2)
        # 単体に「メラ」を唱える魔法攻撃
        if magic_types == 1:
            return self.common_magic(attack_target, attack_message, 2, 2, "メラ", "single")
        # 全体に「イオ」を唱える魔法攻撃
        elif magic_types == 2:
            return self.common_magic(attack_target, attack_message, 5, 4, "イオ", "all")


# 3.holy魔法使い（回復）
class WizardB(Player):
    # 回復魔法(ホイミかベホマラー)を決める
    def attack_type2(self, attack_target, heal_target, attack_message, heal_strategy):
        #magic_types = int(input("1：ホイミ、2：ベホマラー　選択→"))
        magic_types = random.randint(1,2)
        # 単体に「ホイミ」を唱える魔法回復
        if magic_types == 1:
            return self.common_select_heal_target(heal_target, attack_message, heal_strategy,3, 6, "ホイミ", "single")
        # 全体に「ベホマラー」を唱える魔法回復を定義
        elif magic_types == 2:
            return self.common_select_heal_target(heal_target, attack_message, heal_strategy, 10, 6, "ベホマラー","all")
      # 魔法使い（回復）の攻撃処理おわり



# モンスターの初期データ（名前・HP・MPなど）を設定
m_dorakiA    = Doraki ("メイジドラキーA",30, 16, 19, 4)
m_dorakiB    = Doraki ("メイジドラキーB",30, 16, 19, 4)
kimeraA    = Kimera   ("キメラA",        50, 23, 14, 3)
kimeraB    = Kimera   ("キメラB",        50, 23, 14, 3)
h_suraimuA = h_suraimu("ホイミスライムA",40, 14, 30, 5)
h_suraimuB = h_suraimu("ホイミスライムB",40, 14, 30, 5)

# プレイヤーの情報
knight   = Knight ("騎士",        180, 15, 30, 5)
wizardA  = WizardA("fire魔法使い",160,  8, 21, 5)
wizardB  = WizardB("holy魔法使い",140,  6, 39, 5)

# ループ処理のためにモンスターとプレイヤーのリストを作成
monsters = [kimeraA, kimeraB, h_suraimuA, h_suraimuB, m_dorakiA, m_dorakiB]
players  = [knight, wizardA, wizardB]

# ---------------------------------------------------------

# モンスターの出現情報
for monster in monsters:
    print(f"{monster.name}が現れた。")

turn = 0 # ターン数表示のため
while players and monsters:   # リストにプレイヤーもモンスターもいる時は繰り返す（monstersと書くだけで誰かがいることを指す）
    turn += 1
    print(f"\n●{turn}ターン目●")

    # プレイヤー全員の攻撃
    if players[0].attack_start(monsters, players,  "を倒した。", "lowest", players, monsters) :
        break # attack_startでTrueなら終わる
    # モンスター全員の攻撃
    if players[0].attack_start(players, monsters,"は気絶した＞＜。。", "random", players, monsters):
        break



In [ ]:
# Character（親クラス：共通機能）
# │　 ①プレイヤー/モンスターの攻撃と生存判定：attack_start　→　②へ
# │　 ②攻撃方法選択（物理攻撃か他攻撃か決める）：select_attack_type
# │　　　　※物理攻撃だった時は③-1へ　　（それ以外は各キャラクターへ：attack_type2）
# │
# │   ③-1物理攻撃ターゲットを決める:common_select_attack_target
# │　 ③-2物理攻撃：common_attack
# │
# │　 ④魔法攻撃：common_magic(単体も全体も)
# │
# │   ⑤-1回復ターゲットを決める:common_select_heal_target
# │　 ⑤-2回復魔法：common_hoimi(単体も全体も)
# │
# ├──Monster（モンスター共通）
# │　　 ├──メイジドラキー
# │　 　│　 　└──攻撃魔法（単体）ヒャド　→　④へ
# │　 　│
# │　　 ├──キメラ
# │　 　│　 　└──攻撃魔法（全体）ギラ　→　④へ
# │　 　│
# │　　 └──ホイミスライム
# │　 　  　 　└──回復魔法（ホイミ）　→　⑤-1へ
# │
# └──Player（プレイヤー共通）
#   　　 ├──騎士
# 　　　 │　 　└──回復魔法（ホイミ）　→　⑤-1へ
# 　　　 │
#   　　 ├──fire魔法使い（攻撃）
# 　　　 │　 　└──攻撃魔法
# 　　 　│　　        ├──攻撃魔法（単体）メラ　→　④へ
# 　　 　│ 　　       └──攻撃魔法（全体）イオ　→　④へ
# 　　 　│
# 　　   └──holy魔法使い（回復）
# 　　　  　  　└──回復魔法
# 　　　  　  　       ├──回復魔法（ホイミ）　　→　⑤-1へ
# 　　　  　  　       └──回復魔法（ベホマラー）→　⑤-1へ